In [1]:
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
from scipy import stats
import re
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from myplots import roundup, rounddown, find_decimal_fold, percentile_cut_off, rarefaction_calc, rarefaction_plot
from matplotlib.ticker import FormatStrFormatter
import cPickle as pickle
from Bio.SeqUtils import GC
import seaborn as sns

# finding the correct registration codes using Daphna's functions

###### PNPChip.QC.getQCedMatchRegNum module from Daphna
I copied it all to here because I had problems with the eclipse:

In [ ]:
import pandas
from _mysql_exceptions import Error
from Utils import Load
resultsExcel='/net/mraid08/export/jafar/Microbiome/Analyses/Daphna/PNPChip/tmpnonImputedChipFilesMAF_Single_StoolStoolVsChipAnalysis.xlsx'
matchingResultsExcel=pandas.read_excel(resultsExcel,sheetname='All')
matchingResultsExcel.index=matchingResultsExcel.index.astype('str')
matchingResultsExcel.correctStool_id=matchingResultsExcel.correctStool_id#.astype('str')#.apply(lambda x: str(x).split('.')[0])
matchingResultsExcel.batch_winners=matchingResultsExcel.batch_winners.apply(eval)
resolved=['1','3!','6!!']
multipleStoolForRegnum=Load('/net/mraid08/export/jafar/Microbiome/Analyses/Daphna/PNPChip/MultipleStoolForRegTableAllSamples.dat')
locationAndDate=Load('/net/mraid08/export/genie/Microbiome/Analyses/Daphna/PNPChip/ParticipantsDateLocationInFridgeWithDates.dat').set_index('regNum')
import numpy as np
def resolvedType(conf):
    return conf[0] in resolved or\
    conf[:2] in resolved or\
    conf[:3] in resolved
    
def fillCorrectStoolID(df):
    df = df.reset_index()
    df.loc[df['correctStool_id'].notnull(),'correctStool_id']=df.loc[df['correctStool_id'].notnull(),'correctStool_id'].\
                                                                apply(lambda x: str(x).split('.')[0]).astype('str')
    df.loc[df['correctStool_id'].isnull(),'correctStool_id'] = df.loc[df['correctStool_id'].isnull(),'index'].\
                                                                apply(lambda x: str(x).split('@')[0]).astype('str')
    df=df.set_index("correctStool_id")
    return df

def participantHasNoStool_and_HisBlood_wasnt_assigned_to_someone_else(regNum,df):
    winners=sum(df[df['resolved']].batch_winners.tolist(),[])
    if regNum not in df.index and regNum not in winners:
        return True
    else:
        return False

def returnType(func):
    def wrapFunc(regNum):
        returnType=type(regNum)
        if returnType!=str:
            returnType=int
        regNum=str(regNum).split(".")[0]
        res=func(regNum)
        typeres=returnType(str(res).split(".")[0]) if res is not None and res!='nan' else None
#         if type(typeres)!=int:
#             raise Error
        return typeres
    return wrapFunc

@returnType
def getQCedBloodRegNumByRegNum(regNum):
    # tested in test_QCgetters 
    df = matchingResultsExcel
    df =fillCorrectStoolID(df)
    df['resolved']=df.confidenceBatch.apply(resolvedType)
    if participantHasNoStool_and_HisBlood_wasnt_assigned_to_someone_else(regNum,df):
        return regNum
    winners=df[df['resolved']].batch_winners
    if regNum in winners.index:
        match=winners.loc[regNum]
        if type(match)==list:
            summed_match=match
        else:
            summed_match = list(set(sum(match.values,[]))) 
        if len(summed_match)==1:
            return summed_match[0]
        else:
            raise Exception("Bad resolving, conflicting blood label for DB_ID %s"%regNum)
    return None

def getLocationInFridge(regnum):
    correctRegNum=getQCedBloodRegNumByRegNum(regnum)
    return locationAndDate.loc[correctRegNum]
@returnType
def getQCedStoolRegNumByRegNum(regNum):
    df = matchingResultsExcel
    df =fillCorrectStoolID(df)
    if regNum in df.index.values:
        if type(df.loc[regNum]['index'])==str:
            return df.loc[regNum]['index']
        else:
            res= set([val.split('@')[0] for val in list(set(df.loc[regNum]['index']))])
#             try:
#                 assert len(res)==1
#             except:
#                 print regNum
            res = list(res)[0]
            return res
    if regNum in df["index"].values:
        return None
    return regNum

def getFDbySample(sampleID):
    return 'FD'+multipleStoolForRegnum[multipleStoolForRegnum['outputRegNum']==sampleID.replace('@','_')]\
            .index.values[0].split('FD')[1].split('.')[0]
            

def getQCedAllStoolRegNumByRegNum(regNum):
    df = matchingResultsExcel
    df =fillCorrectStoolID(df).reset_index().set_index('index')
    
    df_reindexed = df.reset_index()
    stool_ids = list(df_reindexed[df_reindexed['correctStool_id']==regNum]['index'].values)
    stool_ids = list(set([x + '@0' if '@' not in x else x for x in stool_ids]))
    stool_fds = [getFDbySample(x) for x in stool_ids]
    print zip(stool_ids,stool_fds)
    return stool_fds


def getFDForBadStool():
    signatureDF = pandas.read_excel('/net/mraid08/export/jafar/Microbiome/Analyses/Daphna/PNPChip/EladDaphaEran_Approved_and_Overriden.xlsx')
    badStoolIds=signatureDF[signatureDF["correctStool_id"]=='X']['stool_id'].values
    multipleStoolForRegnum=Load('/net/mraid08/export/jafar/Microbiome/Analyses/Daphna/PNPChip/MultipleStoolForRegTableAllSamples.dat')
    fdnames=[]
    for badStool in badStoolIds:
        if '@' in str(badStool):
            regnum=badStool.split('@')[0]
            fdname='FD'+multipleStoolForRegnum[multipleStoolForRegnum['outputRegNum']==badStool.replace('@','_')]\
            .index.values[0].split('FD')[1].split('.')[0]
        else:
            fdname='FD'+multipleStoolForRegnum[multipleStoolForRegnum['RegNum']==str(badStool)]\
            .index.values[0].split('FD')[1].split('.')[0]
        fdnames.append(fdname)
    return fdnames

if __name__=="__main__":
    getFDForBadStool()
    regNums=pandas.read_csv('/net/mraid08/export/jafar/Microbiome/Analyses/PNPChip/rawData/phenotypes/regNums.csv')
    regNums['registrationCode']=regNums['registrationCode'].apply(str) 
    regNums['correctStoolRegistrationCode']=regNums.registrationCode.apply(getQCedStoolRegNumByRegNum)
    regNums['correctBloodRegistrationCode']=regNums.registrationCode.apply(getQCedBloodRegNumByRegNum)
    print getQCedBloodRegNumByRegNum('3')
    

In [ ]:
## this is the function Daphna told me to use, but I have to run the whole module above everytime again
## before running the function again:

locationAndDate=getLocationInFridge(987069)

In [ ]:
## tried to generate a loop for this problematic function:...


reg_list=[738601,450464,127686,134681]

for reg in reg_list:
    locationAndDate=getLocationInFridge(reg)
    DNAsample=str(locationAndDate['BD'])
    print reg
    print DNAsample
    

In [ ]:
# generating a list of all original registration codes:

reg_list_3=list(BloodDNASamples_DBExtractedOn200717['RegistrationCode'])
reg_list_4=list(set(reg_list_3))

In [ ]:
print len(reg_list_3)
print len(reg_list_4)

In [ ]:
from PNP_QC.getQCedMatchRegNum import getQCedBloodRegNumByRegNum

In [ ]:
## a loop to find the correct reg code for each original reg code:

count=0
correct_reg_list=[]
CorrectionStatus_list=[]
only_corrected=[]
for reg in reg_list_4:
    Correct=getQCedBloodRegNumByRegNum(reg)
    
    if Correct==reg:
        CorrectionStatus_list.append('same')
        correct_reg_list.append(Correct)
    else:
        count=count+1
        if Correct==None:
            CorrectionStatus_list.append('correct reg unknown')
            correct_reg_list.append('unknown')
        else:
            CorrectionStatus_list.append('corrected')
            correct_reg_list.append(Correct)
            only_corrected.append(Correct)
      
    
        
        
CorrectRegDF=pd.DataFrame({'original registration code': reg_list_4, 
                           'correct registration code':correct_reg_list,
                           'correction status (reg number)': CorrectionStatus_list })

CorrectRegDF.head()


In [ ]:
CorrectRegDF[CorrectRegDF['correction status (reg number)'].isnull()]

In [ ]:
grouping_count=CorrectRegDF.groupby('correction status (reg number)')['correct registration code'].count()
groupingCountsDF=pd.DataFrame({'counts':grouping_count})
groupingCountsDF['percentage']=(groupingCountsDF['counts'])*100/groupingCountsDF['counts'].sum()
groupingCountsDF

In [ ]:
## saving the correct Reg table to pickles and excel:

with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Old Computer Files/Sample files/CorrectRegDF',"wb" ) as f:
    pickle.dump(CorrectRegDF,f)
f.close()


writer='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Old Computer Files/Sample files/CorrectRegDF.xlsx'
CorrectRegDF.to_excel(writer, sheet_name='Sheet1', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, startrow=0, startcol=0, engine=None, merge_cells=True, encoding=None, inf_rep='inf', verbose=True)

## generating comprehensive list of DNA samples extracted from blood
updated to 09/7/17

downloading the list of blood samples extracted from the DB using the following SQL script:

SELECT pnp.users.UserID, pnp.users.RegistrationCode, Lab.blood.UserID, 
Lab.blood.bid1, Lab.blood.bid2 

FROM pnp.users
LEFT JOIN Lab.blood ON pnp.users.UserID = Lab.blood.UserID


In [ ]:
file_name='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Old Computer Files/Sample files/BloodSamples_DBExtractedOn200717.xlsx'
BloodSamples_DBExtractedOn200717=pd.read_excel(file_name)
BloodSamples_DBExtractedOn200717.head()


In [ ]:
## removing samples with neither bid1 nor bid2, and dropping the UserID columns:

CleanBloodSamples=BloodSamples_DBExtractedOn090717[(BloodSamples_DBExtractedOn090717['bid1'].notnull())|
                                                  (BloodSamples_DBExtractedOn090717['bid2'].notnull())]
CleanBloodSamples=CleanBloodSamples.drop(['UserID','UserID.1'], axis=1)
CleanBloodSamples


#downloading the list of blood DNA samples extracted from the DB using the following SQL script:

SELECT pnp.users.UserID, pnp.users.RegistrationCode, Lab.hostdna.UserID,
Lab.hostdna.DnaID

FROM pnp.users
LEFT JOIN Lab.hostdna ON pnp.users.UserID = Lab.hostdna.UserID


In [ ]:
file_name='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BloodDNASamples_DBExtractedOn200717.xlsx'
BloodDNASamples_DBExtractedOn200717=pd.read_excel(file_name)
BloodDNASamples_DBExtractedOn200717.head()


In [ ]:
# drop all rows that don't have BD num
BloodDNASamplesNoNan=BloodDNASamples_DBExtractedOn200717[~BloodDNASamples_DBExtractedOn200717['DnaID'].isnull()]



BloodDNASamplesNoNan

### downloading the correct reg number table that I generated using Daphna's function:





In [ ]:
file_name='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/CorrectRegDF.xlsx'
CorrectRegDF=pd.read_excel(file_name)
CorrectRegDF.head()

In [ ]:
CorrectBloodDNASamples=pd.merge(BloodDNASamplesNoNan,CorrectRegDF,how='left',
                                    left_on='RegistrationCode',right_on='original registration code')
CorrectBloodDNASamples.head()

In [ ]:
## make clean file with no wrong reg numbers and user IDs:
CleanCorrectBloodDNASamples=CorrectBloodDNASamples.drop(['RegistrationCode','UserID.1'],axis=1)
CleanCorrectBloodDNASamples.head()


In [ ]:
CleanCorrectBloodDNASamples['DnaIndex']=CleanCorrectBloodDNASamples['DnaID'].str.replace("BD","")

In [ ]:
CleanCorrectBloodDNASamples['DnaIndex']=CleanCorrectBloodDNASamples['DnaIndex'].astype(int)

In [ ]:
CleanCorrectBloodDNASamples.head()

In [ ]:
mask = CleanCorrectBloodDNASamples['DnaIndex']> 791
column_name_to_replace = 'correct registration code'
column_name_to_replaceWith = 'original registration code'
CleanCorrectBloodDNASamples.loc[mask, column_name_to_replace] = CleanCorrectBloodDNASamples.loc[mask, column_name_to_replaceWith]

In [ ]:
mask = CleanCorrectBloodDNASamples['DnaIndex']> 791
column_name_to_replace = 'correction status (reg number)'
CleanCorrectBloodDNASamples.loc[mask, column_name_to_replace] = 'RegNUM not checked'

In [ ]:
CleanCorrectBloodDNASamples[CleanCorrectBloodDNASamples['DnaIndex']>791]

In [ ]:
CleanCorrectBloodDNASamples

In [ ]:
CleanCorrectBloodDNASamples.groupby('correction status (reg number)')['DnaID'].count()

In [ ]:
## saving the CleanCorrectBloodDNASamples table to pickles and excel:

with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/CleanCorrectBloodDNASamples',"wb" ) as f:
    pickle.dump(CleanCorrectBloodDNASamples,f)
f.close()

writer='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/CleanCorrectBloodDNASamples.xlsx'
CleanCorrectBloodDNASamples.to_excel(writer, sheet_name='Sheet1', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, startrow=0, startcol=0, engine=None, merge_cells=True, encoding=None, inf_rep='inf', verbose=True)


In [ ]:
## merge the clean blood DNA sample with the blood sample based on correct REG number:
BloodDNASamples200717correctREG=pd.merge(CleanCorrectBloodDNASamples,CleanBloodSamples, how='left',
                                        left_on='correct registration code',right_on='RegistrationCode')
BloodDNASamples200717correctREG=BloodDNASamples200717correctREG.drop('RegistrationCode',axis=1)
BloodDNASamples200717correctREG.head()

In [ ]:
## downloading file with usersID and regCode:
file_name='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Old Computer Files/Sample files/UserIDs_and_regNums_110717.xlsx'
UserIDs_and_regNums_110717=pd.read_excel(file_name)
UserIDs_and_regNums_110717.head()

In [ ]:
#add user IDs to the blood DNA file:
BloodDNASamples200717correctREGwithUserID=pd.merge(BloodDNASamples090717correctREG,UserIDs_and_regNums_110717, how='left',
                                        left_on='correct registration code',right_on='RegistrationCode')

BloodDNASamples200717correctREGwithUserID.head()


In [ ]:
# remove redundent reg code column:
BloodDNASamples200717correctREGwithUserID=BloodDNASamples200717correctREGwithUserID.drop('RegistrationCode', axis=1)
BloodDNASamples200717correctREGwithUserID.head()

In [ ]:
#downloading those who signed helsinki:



file_name='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Old Computer Files/Sample files/signedHelsinki (1).xlsx'
signedHelsinki=pd.read_excel(file_name)
signedHelsinki.head()





In [ ]:
BloodDNASamples200717=pd.merge(BloodDNASamples090717correctREGwithUserID,signedHelsinki, how='left',
                                    left_on='UserID',right_on='UserID')
BloodDNASamples200717.head()

In [ ]:
BloodDNASamples200717[BloodDNASamples200717['SignedGenHels']==0]

In [ ]:
print len(BloodDNASamples200717[BloodDNASamples200717['SignedGenHels']!=1])

In [ ]:
# remove duplicates:

BloodDNASamples200717_2=BloodDNASamples200717.drop_duplicates()

In [ ]:
print len(BloodDNASamples200717)
print len(BloodDNASamples200717_2)

In [ ]:
grouped_future_bids=BloodDNASamples200717.groupby('DnaID')['bid1'].apply(list)
grouped_future_bids=grouped_future_bids.reset_index()
grouped_future_bids[50:100]


In [ ]:
print len(grouped_future_bids)

In [ ]:
# MAKE THE BID1 column a summary of all bid1 samples per DNA sample:


BloodDNASamples200717_3=pd.merge(BloodDNASamples200717_2,grouped_future_bids, how='left', 
                                    left_on='DnaID', right_on='DnaID')
BloodDNASamples200717_3=BloodDNASamples200717_3.rename(columns={'bid1_y':'all bid1s'}).drop('bid1_x', axis=1)
BloodDNASamples200717_3.head()

In [ ]:
# remove duplicate lines for the same DNA sample:

BloodDNASamples200717_4=BloodDNASamples200717_3.drop_duplicates(subset='DnaID')
print len(BloodDNASamples200717_4)
print len(BloodDNASamples200717_3)

In [ ]:

BloodDNASamples200717_4.head()

## this is the final table for samples extracted from the database

this table can be merged with each updated file from Tali

Save the final table:

In [ ]:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Old Computer Files/Sample files/BloodDNASamples110717_4',"wb" ) as f:
    pickle.dump(BloodDNASamples110717_4,f)
f.close()


writer='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Old Computer Files/Sample files/BloodDNASamples110717_4.xlsx'
BloodDNASamples110717_4.to_excel(writer, sheet_name='Sheet1', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, startrow=0, startcol=0, engine=None, merge_cells=True, encoding=None, inf_rep='inf', verbose=True)

## merge the correct blood and DNA samples with Tali's table:

In [ ]:
#downloading samples that Tali already cleaned with their updated concentrations

file_name='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Old Computer Files/Sample files/CleaningInfoFromTaliShort180717.xlsx'
CleanedSamplesOnlyNamesConc=pd.read_excel(file_name)
CleanedSamplesOnlyNamesConc.head()



In [ ]:
print len(CleanedSamplesOnlyNamesConc)



In [ ]:
## merge the correct blood and DNA samples with Tali's table:

BloodDNASamplesCleaningInfoUpdated200717=pd.merge(BloodDNASamples200717_4,CleanedSamplesOnlyNamesConc,how='outer',
                                    left_on='DnaID',right_on='BD')
BloodDNASamplesCleaningInfoUpdated200717.head()

In [ ]:
print len(BloodDNASamplesCleaningInfoUpdated200717)

In [ ]:
mask = BloodDNASamplesCleaningInfoUpdated200717['#']> 875
column_name_to_replace = 'DnaID'
column_name_to_replaceWith = 'BD'
BloodDNASamplesCleaningInfoUpdated200717.loc[mask, column_name_to_replace] = BloodDNASamplesCleaningInfoUpdated200717.loc[mask, column_name_to_replaceWith]

In [ ]:
mask = BloodDNASamplesCleaningInfoUpdated200717['#']> 875
column_name_to_replace = 'DnaIndex'
column_name_to_replaceWith = '#'
BloodDNASamplesCleaningInfoUpdated200717.loc[mask, column_name_to_replace] = BloodDNASamplesCleaningInfoUpdated200717.loc[mask, column_name_to_replaceWith]

In [ ]:
BloodDNASamplesCleaningInfoUpdated200717[BloodDNASamplesCleaningInfoUpdated200717['#']> 875]

In [ ]:
BloodDNASamplesCleaningInfoUpdated200717.set_index('DnaIndex')

In [ ]:
BloodDNASamplesCleaningInfoUpdated200717_2=BloodDNASamplesCleaningInfoUpdated200717.drop(['#','BD'], axis=1)

In [ ]:
BloodDNASamplesCleaningInfoUpdated200717_2.head()

downloading and merging BDPure information:

In [ ]:
file_name='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Old Computer Files/Sample files/Copy of Sum_Blood_DNA_Concentration2.xlsx'
BDPureInfo=pd.read_excel(file_name)

BDPureInfo.head()

In [ ]:
BloodDNASamplesCleaningInfoUpdated200717_3=pd.merge(BloodDNASamplesCleaningInfoUpdated200717_2,BDPureInfo,how='left',
                                    left_on='DnaID',right_on='Sample ID')
BloodDNASamplesCleaningInfoUpdated200717_3.head()

In [ ]:
BloodDNASamplesCleaningInfoUpdated200717_3.drop('Sample ID',axis=1, inplace=True)

BloodDNASamplesCleaningInfoUpdated200717_3.head()

In [ ]:
BloodDNASamplesCleaningInfoUpdated200717_3.set_index('DnaIndex',inplace=True)

In [ ]:
BloodDNASamplesCleaningInfoUpdated200717_3['enough for 0.5ug']=np.where(BloodDNASamplesCleaningInfoUpdated200717_3[
        'ng/ul after cleanUp']>32.25,1,0)

In [ ]:
BloodDNASamplesCleaningInfoUpdated200717_3['need to re-clean']=np.where((BloodDNASamplesCleaningInfoUpdated200717_3[
        'ng/ul after cleanUp']<32.25)&(BloodDNASamplesCleaningInfoUpdated200717_3['ng/ul after cleanUp'].notnull()),1,0)

In [ ]:
a=BloodDNASamplesCleaningInfoUpdated200717_3.loc[109,'all bid1s']
a

## need to rewrite the condition for the 'need to rextract' column


In [ ]:
BloodDNASamplesCleaningInfoUpdated200717_3['need to re-extract DNA from blood']=np.where(condition,1,0)

In [ ]:
BloodDNASamplesCleaningInfoUpdated200717_3[BloodDNASamplesCleaningInfoUpdated200717_3['ng/ul after cleanUp'].isnull()].head()

In [ ]:
SamplesToReextract=BloodDNASamplesCleaningInfoUpdated200717_3[BloodDNASamplesCleaningInfoUpdated200717_3['need to re-extract DNA from blood']==1]

In [ ]:
SamplesToReClean=BloodDNASamplesCleaningInfoUpdated200717_3[BloodDNASamplesCleaningInfoUpdated200717_3['need to re-clean']==1]

In [ ]:
print len(SamplesToReextract)

In [ ]:
SamplesToReClean

In [ ]:
print len(SamplesToReClean)

In [ ]:
print len(BloodDNASamplesCleaningInfoUpdated200717_3)

In [ ]:
BloodDNASamplesCleaningInfoUpdated200717_3.sort_index(inplace=True)
BloodDNASamplesCleaningInfoUpdated200717_4=BloodDNASamplesCleaningInfoUpdated200717_3.drop_duplicates('DnaID')

In [ ]:
print len(BloodDNASamplesCleaningInfoUpdated200717_4)

In [ ]:
774-241

In [ ]:
533./948

In [ ]:
## saving:

with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Old Computer Files/Sample files/BloodDNASamplesCleaningInfoUpdated200717_4',"wb" ) as f:
    pickle.dump(BloodDNASamplesCleaningInfoUpdated200717_4,f)
f.close()

writer='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Old Computer Files/Sample files/BloodDNASamplesCleaningInfoUpdated200717_4.xlsx'
BloodDNASamplesCleaningInfoUpdated200717_4.to_excel(writer, sheet_name='Sheet1', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, startrow=0, startcol=0, engine=None, merge_cells=True, encoding=None, inf_rep='inf', verbose=True)

writer='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Old Computer Files/Sample files/SamplesToReextract.xlsx'
SamplesToReextract.to_excel(writer, sheet_name='Sheet1', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, startrow=0, startcol=0, engine=None, merge_cells=True, encoding=None, inf_rep='inf', verbose=True)

writer='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Old Computer Files/Sample files/SamplesToReClean.xlsx'
SamplesToReClean.to_excel(writer, sheet_name='Sheet1', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, startrow=0, startcol=0, engine=None, merge_cells=True, encoding=None, inf_rep='inf', verbose=True)


In [ ]:
print len(BloodDNASamplesCleaningInfoUpdated200717_4[BloodDNASamplesCleaningInfoUpdated200717_4['ng/ul after cleanUp'].notnull()])

In [ ]:
774.0/948


## adding UserIDs for new samples and adding  study type to all:

In [ ]:
file_name='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Old Computer Files/Sample files/BDnumber_and_UserIDs_someAreWrong.xlsx'
BDnumber_and_UserIDs_someAreWrong=pd.read_excel(file_name)
BDnumber_and_UserIDs_someAreWrong.head()

In [ ]:
BloodDNASamplesCleaningInfoUpdated200717_4.head()

In [ ]:
BloodDNASamplesCleaningInfoUpdated200717_5=pd.merge(BloodDNASamplesCleaningInfoUpdated200717_4,BDnumber_and_UserIDs_someAreWrong,how='left',
                                                   left_on='DnaID', right_on='DnaID')

In [ ]:
BloodDNASamplesCleaningInfoUpdated200717_5['UserIDnew']=np.where(BloodDNASamplesCleaningInfoUpdated200717_5['UserID_x'].isnull(),
                                                                BloodDNASamplesCleaningInfoUpdated200717_5['UserID_y'],
                                                                BloodDNASamplesCleaningInfoUpdated200717_5['UserID_x'])


In [ ]:
BloodDNASamplesCleaningInfoUpdated200717_5[850:900]

In [ ]:
BloodDNASamplesCleaningInfoUpdated200717_6=BloodDNASamplesCleaningInfoUpdated200717_5.rename(columns={'UserIDnew':'UserID'})

In [ ]:
BloodDNASamplesCleaningInfoUpdated200717_6=BloodDNASamplesCleaningInfoUpdated200717_6.drop(['UserID_x', 'UserID_y'], axis=1)

In [ ]:
print len(BloodDNASamplesCleaningInfoUpdated200717_6)

In [ ]:
file_name='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Old Computer Files/Sample files/UserIDs_studyType.xlsx'
UserIDs_studyType=pd.read_excel(file_name)
UserIDs_studyType.head()

In [ ]:
BloodDNASamplesCleaningInfoUpdated200717_7=pd.merge(BloodDNASamplesCleaningInfoUpdated200717_6,UserIDs_studyType,how='left',
                                                   left_on='UserID', right_on='UserID')

In [ ]:
BloodDNASamplesCleaningInfoUpdated200717_7[950:1000]

In [ ]:
print len(BloodDNASamplesCleaningInfoUpdated200717_7)

In [ ]:
BloodDNASamplesCleaningInfoUpdated200717_8=BloodDNASamplesCleaningInfoUpdated200717_7.drop_duplicates('DnaID')

In [ ]:
print len(BloodDNASamplesCleaningInfoUpdated200717_8)

In [ ]:
BloodDNASamplesCleaningInfoUpdated200717_8['DnaIDindex']=BloodDNASamplesCleaningInfoUpdated200717_8['DnaID'].str.replace("BD","")

In [ ]:
BloodDNASamplesCleaningInfoUpdated200717_8.head()

In [ ]:
# downloading the lookup table for study types:
file_name='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Old Computer Files/Sample files/studyTypeTable.xlsx'
studyTypeTable=pd.read_excel(file_name)
studyTypeTable.head()

In [ ]:
BloodDNASamplesCleaningInfoUpdated200717_9=pd.merge(BloodDNASamplesCleaningInfoUpdated200717_8,studyTypeTable, how='left',
                                                   left_on='StudyTypeID', right_on='StudyID')
BloodDNASamplesCleaningInfoUpdated200717_9.head()

In [ ]:
#BloodDNASamplesCleaningInfoUpdated200717_9=BloodDNASamplesCleaningInfoUpdated200717_9.drop(['StudyTypeID','StudyID'], axis=1)
BloodDNASamplesCleaningInfoUpdated200717_9.set_index('DnaIDindex', inplace=True)
BloodDNASamplesCleaningInfoUpdated200717_9

In [ ]:
BloodDNASamplesCleaningInfoUpdated200717_9=BloodDNASamplesCleaningInfoUpdated200717_9[BloodDNASamplesCleaningInfoUpdated200717_9['DnaID'].notnull()]

In [ ]:
BloodDNASamplesCleaningInfoUpdated200717_9.head()

In [ ]:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Old Computer Files/Sample files/BloodDNASamplesCleaningInfoUpdated200717_9',"wb" ) as f:
    pickle.dump(BloodDNASamplesCleaningInfoUpdated200717_9,f)
f.close()

writer='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Old Computer Files/Sample files/BloodDNASamplesCleaningInfoUpdated200717_9.xlsx'
BloodDNASamplesCleaningInfoUpdated200717_9.to_excel(writer, sheet_name='Sheet1', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, startrow=0, startcol=0, engine=None, merge_cells=True, encoding=None, inf_rep='inf', verbose=True)

## changes to correction status:
there were reg nmbers reported as unknown by Daphna's function but lately she and Eran and decided to use them as correct numbers since they were not clearly defined as confused with other samples. Daphna gave me the list of all reg numbers that are considered to be correct and I need to cross it with my list and update this samples:

In [ ]:
# load the most updated sample list that I have
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BloodDNASamplesCleaningInfoUpdated200717_9'
BloodDNASamplesCleaningInfoUpdated200717_9=pd.read_pickle(file1)


In [ ]:
#load the list from Daphan:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/correct_genotyped_registrationNumbers_fromDaphna200817.xlsx'
CorrectRegNumsUpdated=pd.read_excel(file1)

In [ ]:
CorrectRegNumsUpdated.head()

In [ ]:
BloodDNASamplesCleaningInfoUpdated200817=pd.merge(BloodDNASamplesCleaningInfoUpdated200717_9,CorrectRegNumsUpdated,how='outer',
                                                  left_on='original registration code', right_on=1,copy=False)

In [ ]:
#load the blood sample file extracted from the DB ON 200717::
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BloodDNASamples_DBExtractedOn200717.xlsx'
BloodDNASamples_DBExtractedOn200717=pd.read_excel(file1)



In [ ]:
BloodDNASamplesCleaningInfoUpdated200817_3=pd.merge(BloodDNASamplesCleaningInfoUpdated200717_9,BloodDNASamples_DBExtractedOn200717,
                                                   how='inner', left_on='DnaID',right_on='DnaID',copy=False)

In [ ]:
print len(BloodDNASamplesCleaningInfoUpdated200817_3)

In [ ]:
BloodDNASamplesCleaningInfoUpdated200817_3.head()

In [ ]:
BloodDNASamplesCleaningInfoUpdated200817_3['UserID_x']=np.where(BloodDNASamplesCleaningInfoUpdated200817_3['UserID_x'].isnull(),
                                                               BloodDNASamplesCleaningInfoUpdated200817_3['UserID_y'],
                                                               BloodDNASamplesCleaningInfoUpdated200817_3['UserID_x'])

In [ ]:
BloodDNASamplesCleaningInfoUpdated200817_3['correct registration code']=np.where(BloodDNASamplesCleaningInfoUpdated200817_3['correct registration code'].isnull(),
                                                               BloodDNASamplesCleaningInfoUpdated200817_3['RegistrationCode'],
                                                               BloodDNASamplesCleaningInfoUpdated200817_3['correct registration code'])

In [ ]:
BloodDNASamplesCleaningInfoUpdated200817_3['original registration code']=np.where(BloodDNASamplesCleaningInfoUpdated200817_3['original registration code'].isnull(),
                                                               BloodDNASamplesCleaningInfoUpdated200817_3['RegistrationCode'],
                                                               BloodDNASamplesCleaningInfoUpdated200817_3['original registration code'])

In [ ]:
BloodDNASamplesCleaningInfoUpdated200817_3['correction status (reg number)']=np.where(BloodDNASamplesCleaningInfoUpdated200817_3['correction status (reg number)'].isnull(),
                                                               'RegNUM not checked',
                                                               BloodDNASamplesCleaningInfoUpdated200817_3['correction status (reg number)'])

In [ ]:
BloodDNASamplesCleaningInfoUpdated200817_3=BloodDNASamplesCleaningInfoUpdated200817_3.drop(['UserID_y','RegistrationCode','UserID.1'],axis=1)

In [ ]:
BloodDNASamplesCleaningInfoUpdated200817_3=BloodDNASamplesCleaningInfoUpdated200817_3.rename(columns={'UserID_x':'UserID'})

In [ ]:
BloodDNASamplesCleaningInfoUpdated200817_3[BloodDNASamplesCleaningInfoUpdated200817_3['correction status (reg number)']!='same']

In [ ]:
writer='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/NotCorrectedRegNums.xlsx'
NotCorrectedRegNums.to_excel(writer, sheet_name='Sheet1', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, startrow=0, startcol=0, engine=None, merge_cells=True, encoding=None, inf_rep='inf', verbose=True)

In [ ]:
CorrectRegNumsUpdatedList=list(CorrectRegNumsUpdated[1])

In [ ]:
for i in BloodDNASamplesCleaningInfoUpdated200817_3.index:
    if BloodDNASamplesCleaningInfoUpdated200817_3.loc[i,'correct registration code'] in CorrectRegNumsUpdatedList:
        BloodDNASamplesCleaningInfoUpdated200817_3.loc[i,'genotyped']=1
    else:
         BloodDNASamplesCleaningInfoUpdated200817_3.loc[i,'genotyped']=0

In [ ]:
BloodDNASamplesCleaningInfoUpdated200817_3[(BloodDNASamplesCleaningInfoUpdated200817_3['genotyped']!=1)&(BloodDNASamplesCleaningInfoUpdated200817_3['correction status (reg number)']=='corrected')]

In [ ]:
BloodDNASamplesCleaningInfoUpdated200817_3=BloodDNASamplesCleaningInfoUpdated200817_3.rename(columns={'SampleInDaphnasList':'RegCodeInDaphnasList'})

In [ ]:
BloodDNASamplesCleaningInfoUpdated200817_3[(BloodDNASamplesCleaningInfoUpdated200817_3['genotyped']!=0)&(BloodDNASamplesCleaningInfoUpdated200817_3['correction status (reg number)']=='correct reg unknown')]

In [ ]:
#GENERATING A LIST OF SAMPLES THAT WERE GENOTYPED:

In [ ]:
sample_list=[]

In [ ]:
sample_list

In [ ]:
folder_list

In [ ]:
folder_list=['Plate1','Plate2','Plate3','Plate4','Plate5-9']

sample_list=[]
nfiles=0
for folder in folder_list:
    curr_folder='/net/mraid08/export/jafar/Microbiome/Data/PNP/GeneByGeneData/%s' %folder
    filenames = [f for f in listdir(curr_folder) if isfile(join(curr_folder, f))]
    nfiles1=len(filenames)
    nfiles=nfiles+nfiles1

    
    for file_name in filenames:
        newf1=re.sub("\w+\-\w+BD", "BD", file_name)
        newf2=newf1.replace('.txt','')
        sample_list.append(newf2)
        
anotherfolder='/net/mraid08/export/jafar/Microbiome/Data/PNP/GeneByGeneData/Batch1442_MCM_1_1_Segal'

filenames2 = [f for f in listdir(anotherfolder) if isfile(join(anotherfolder, f))]

for file_name2 in filenames2:
    nfiles2=len(filenames2)
    nfiles=nfiles+nfiles2
    newf1=re.sub("\w+\-BD", "BD", file_name2)
    newf2=newf1.replace('.txt','')
    sample_list.append(newf2)
    


In [ ]:
print nfiles

In [ ]:
print len(sample_list)

In [ ]:
anotherfolder='/net/mraid08/export/jafar/Microbiome/Data/PNP/GeneByGeneData/Batch1442_MCM_1_1_Segal'

filenames2 = [f for f in listdir(anotherfolder) if isfile(join(anotherfolder, f))]
filenames2[0]

In [ ]:
newf1=re.sub("\w+\-BD", "BD", filenames2[0])
newf2=newf1.replace('.txt','')
newf2

In [ ]:
print nfiles

In [ ]:
print len(sample_list)

In [ ]:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/genotypedSamplesInFolders','wb') as f:
       pickle.dump(sample_list,f)
f.close()





In [ ]:
for i in BloodDNASamplesCleaningInfoUpdated200817_3.index:
    if BloodDNASamplesCleaningInfoUpdated200817_3.loc[i,'DnaID'] in sample_list:
        BloodDNASamplesCleaningInfoUpdated200817_3.loc[i,'DnaIDinGenotypingFolder']=1
    else:
         BloodDNASamplesCleaningInfoUpdated200817_3.loc[i,'DnaIDinGenotypingFolder']=0

In [ ]:
BloodDNASamplesCleaningInfoUpdated200817_3

In [ ]:
BloodDNASamplesCleaningInfoUpdated200817_3[(BloodDNASamplesCleaningInfoUpdated200817_3['DnaIDinGenotypingFolder']==1)&(BloodDNASamplesCleaningInfoUpdated200817_3['RegCodeInDaphnasList']==0)]

In [ ]:
checkWithDaphna=BloodDNASamplesCleaningInfoUpdated200817_3[(BloodDNASamplesCleaningInfoUpdated200817_3['DnaIDinGenotypingFolder']==1)&(BloodDNASamplesCleaningInfoUpdated200817_3['RegCodeInDaphnasList']==0) &((BloodDNASamplesCleaningInfoUpdated200817_3['correction status (reg number)']=='corrected')|(BloodDNASamplesCleaningInfoUpdated200817_3['correction status (reg number)']=='same'))]


In [ ]:
checkWithDaphna

In [ ]:
columns_to_keep=['DnaID','correct registration code','correction status (reg number)','original registration code','UserID',
                 'RegCodeInDaphnasList','DnaIDinGenotypingFolder']

In [ ]:
checkWithDaphna=checkWithDaphna[columns_to_keep]

In [ ]:
writer='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/checkWithDaphna.xlsx'
checkWithDaphna.to_excel(writer, sheet_name='Sheet1', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, startrow=0, startcol=0, engine=None, merge_cells=True, encoding=None, inf_rep='inf', verbose=True)

In [ ]:
150+86+92+71+96+388

In [ ]:
print len(BloodDNASamplesCleaningInfoUpdated200817_3[BloodDNASamplesCleaningInfoUpdated200817_3['correction status (reg number)']=='correct reg unknown'])

In [ ]:
for i in BloodDNASamplesCleaningInfoUpdated200817_3.index:
    if (BloodDNASamplesCleaningInfoUpdated200817_3.loc[i,'correction status (reg number)']!='corrected')&\
        (BloodDNASamplesCleaningInfoUpdated200817_3.loc[i,'correction status (reg number)']!='same')&\
         (BloodDNASamplesCleaningInfoUpdated200817_3.loc[i,'DnaIDinGenotypingFolder']==0):
            BloodDNASamplesCleaningInfoUpdated200817_3.loc[i,'correction status (reg number)']='Not Genotyped, assumed to be correct'
            BloodDNASamplesCleaningInfoUpdated200817_3.loc[i,'correct registration code']=BloodDNASamplesCleaningInfoUpdated200817_3.loc[i,'original registration code']

In [ ]:
BloodDNASamplesCleaningInfoUpdated200817_3[BloodDNASamplesCleaningInfoUpdated200817_3['correction status (reg number)']=='Not Genotyped, assumed to be correct']

In [ ]:
BloodDNASamplesCleaningInfoUpdated200817_3[BloodDNASamplesCleaningInfoUpdated200817_3['correction status (reg number)']=='RegNUM not checked']

In [ ]:
for i in BloodDNASamplesCleaningInfoUpdated200817_3.index:
    if (BloodDNASamplesCleaningInfoUpdated200817_3.loc[i,'correction status (reg number)']=='RegNUM not checked')&\
        (BloodDNASamplesCleaningInfoUpdated200817_3.loc[i,'DnaIDinGenotypingFolder']==1)&\
        (BloodDNASamplesCleaningInfoUpdated200817_3.loc[i,'RegCodeInDaphnasList']==0):
            BloodDNASamplesCleaningInfoUpdated200817_3.loc[i,'correction status (reg number)']='note! RegNum probably not correct!'
    elif  (BloodDNASamplesCleaningInfoUpdated200817_3.loc[i,'correction status (reg number)']=='RegNUM not checked')&\
        (BloodDNASamplesCleaningInfoUpdated200817_3.loc[i,'DnaIDinGenotypingFolder']==1)&\
        (BloodDNASamplesCleaningInfoUpdated200817_3.loc[i,'RegCodeInDaphnasList']==1):
            BloodDNASamplesCleaningInfoUpdated200817_3.loc[i,'correction status (reg number)']='same'      

In [ ]:
BloodDNASamplesCleaningInfoUpdated200817_3[BloodDNASamplesCleaningInfoUpdated200817_3['correction status (reg number)']=='Not Genotyped, assumed to be correct']

In [ ]:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BloodDNASamplesCleaningInfoUpdated200817_3',"wb" ) as f:
    pickle.dump(BloodDNASamplesCleaningInfoUpdated200817_3,f)
f.close()

writer='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BloodDNASamplesCleaningInfoUpdated200817_3.xlsx'
BloodDNASamplesCleaningInfoUpdated200817_3.to_excel(writer, sheet_name='Sheet1', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, startrow=0, startcol=0, engine=None, merge_cells=True, encoding=None, inf_rep='inf', verbose=True)

In [ ]:
BloodDNASamplesCleaningInfoUpdated200817_3.groupby('correction status (reg number)').count()['DnaID']

### add Updated Helsinki status:


In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/signedHelsinki (1).xlsx'
signed=pd.read_excel(file1)

In [ ]:
BloodDNASamplesCleaningInfoUpdated200817_4=pd.merge(BloodDNASamplesCleaningInfoUpdated200817_3,signed,how='left', left_on='UserID',
                                                   right_on='UserID')

In [ ]:
print len(BloodDNASamplesCleaningInfoUpdated200817_3)
print len(BloodDNASamplesCleaningInfoUpdated200817_4)

In [ ]:
BloodDNASamplesCleaningInfoUpdated200817_4[BloodDNASamplesCleaningInfoUpdated200817_4.duplicated(subset='DnaID',keep=False)]

In [ ]:
BloodDNASamplesCleaningInfoUpdated200817_4=BloodDNASamplesCleaningInfoUpdated200817_4.drop_duplicates(subset=columns_to_check)

In [ ]:
BloodDNASamplesCleaningInfoUpdated200817_4.columns.values

In [ ]:
columns_to_check=['DnaID', 'correct registration code',
       'correction status (reg number)', 'original registration code',
       'SignedGenHels_x', 'ng/ul after cleanUp',
       'Batch name', 'enough for 0.5ug', 'need to re-clean',
       'need to re-extract DNA from blood', 'UserID', 'StudyName',
       'RegCodeInDaphnasList', 'DnaIDinGenotypingFolder',
       'SignedGenHels_y']

In [ ]:
BloodDNASamplesCleaningInfoUpdated200817_4=BloodDNASamplesCleaningInfoUpdated200817_4.drop('SignedGenHels_x', axis=1)

In [ ]:
BloodDNASamplesCleaningInfoUpdated200817_4=BloodDNASamplesCleaningInfoUpdated200817_4.rename(columns={'SignedGenHels_y':'SignedGenHels'})

In [ ]:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BloodDNASamplesCleaningInfoUpdated200817_4',"wb" ) as f:
    pickle.dump(BloodDNASamplesCleaningInfoUpdated200817_4,f)
f.close()

writer='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BloodDNASamplesCleaningInfoUpdated200817_4.xlsx'
BloodDNASamplesCleaningInfoUpdated200817_4.to_excel(writer, sheet_name='Sheet1', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, startrow=0, startcol=0, engine=None, merge_cells=True, encoding=None, inf_rep='inf', verbose=True)

In [ ]:
wrong=BloodDNASamplesCleaningInfoUpdated200817_4[BloodDNASamplesCleaningInfoUpdated200817_4['correction status (reg number)']=='note! RegNum probably not correct!']

In [ ]:
wrongRegNums=list(wrong['original registration code'])

In [ ]:
## a loop to find the correct reg code for each original reg code:

count=0
correct_reg_list=[]
CorrectionStatus_list=[]
only_corrected=[]
for reg in wrongRegNums:
    Correct=getQCedBloodRegNumByRegNum(reg)
    
    if Correct==reg:
        CorrectionStatus_list.append('same')
        correct_reg_list.append(Correct)
    else:
        count=count+1
        if Correct==None:
            CorrectionStatus_list.append('correct reg unknown')
            correct_reg_list.append('unknown')
        else:
            CorrectionStatus_list.append('corrected')
            correct_reg_list.append(Correct)
            only_corrected.append(Correct)
      
    
        
        
CorrectRegDF200817=pd.DataFrame({'original registration code': wrongRegNums, 
                           'correct registration code':correct_reg_list,
                           'correction status (reg number)': CorrectionStatus_list })

CorrectRegDF200817


## some old stuff:
maybe some of the files or tables are not still valid:

In [ ]:
#generate list of users that didnt sign:

NotSignedGenHel=pd.merge(BloodAndDnaSamplesFromDB,signedHelsinki, how='outer',
                                    left_on='UserID',right_on='UserID')
NotSignedGenHel=NotSignedGenHel[NotSignedGenHel['SignedGenHels']!=1]
NotSignedGenHel=NotSignedGenHel[pd.notnull(
        NotSignedGenHel[['bid1-bloodTableDB','bid2-bloodTableDB','DnaID-hostDnaTableDB']]).any(axis=1)]
NotSignedGenHel.head()
    

In [ ]:
#downloading the blood storage date table:


file_name='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Old Computer Files/Sample files/BloodStorageDT.xlsx'
BloodStorageDT=pd.read_excel(file_name)
BloodStorageDT.head()


In [ ]:
NotSignedGenHel=NotSignedGenHel.drop_duplicates('UserID')

In [ ]:
BloodStorageDT=BloodStorageDT.drop_duplicates('UserID')

In [ ]:
NotSignedGenHel2=pd.merge(NotSignedGenHel,BloodStorageDT, 
                          how='left', left_on='UserID', right_on='UserID')
NotSignedGenHel2.head()

In [ ]:
## saving the not-signed table to pickles and excel:

with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Old Computer Files/Sample files/NotSignedGenHel2',"wb" ) as f:
    pickle.dump(NotSignedGenHel2,f)
f.close()

writer='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Old Computer Files/Sample files/NotSignedGenHel2.xlsx'
NotSignedGenHel2.to_excel(writer, sheet_name='Sheet1', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, startrow=0, startcol=0, engine=None, merge_cells=True, encoding=None, inf_rep='inf', verbose=True)


In [ ]:
groupedByCorrStat=AllBloodDnaSamples130617_9.groupby(['ExpName','correction status (reg number)'])['RegistrationCode'].count()
CorrStatDF=pd.DataFrame({'counts':groupedByCorrStat})
CorrStatDF['percentage']=CorrStatDF['counts']/CorrStatDF['counts'].sum(level=0)
#groupedDFper=pd.DataFrame(groupedDF.groupby(level=1).apply(lambda x: 100 * x / float(x.sum())))

CorrStatDF

In [ ]:
file_name='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Old Computer Files/Sample files/signed list from Noa.xlsx'
SignedNoa=pd.read_excel(file_name)

In [ ]:
NotSignedGenHel2.head()

In [ ]:
signedNoaList=list(SignedNoa['RegCode'])

In [ ]:
count=0
for code in signedNoaList:
    if code in list(NotSignedGenHel2['RegistrationCode']):
        print code
        count+=1
print count

In [ ]:
import PNPChip_QC.getQCedMatchRegNum as pnpchip

In [ ]:
a = pnpchip.getQCedBloodRegNumByRegNum(784215)
a

In [ ]:
import time
cdate=str(time.strftime("%d%m%Y"))
cdate

In [ ]:
writer='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Old Computer Files/Sample files/CorrectRegDF_%s.xlsx' %cdate

In [ ]:
writer

In [ ]:
getQCedBloodRegNumByRegNum(283785)

In [ ]:
a=getQCedBloodRegNumByRegNum(608673)

In [ ]:
a